<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Test-runs" data-toc-modified-id="Test-runs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Test runs</a></span></li><li><span><a href="#Interact" data-toc-modified-id="Interact-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Interact</a></span></li><li><span><a href="#Parameter-sweep" data-toc-modified-id="Parameter-sweep-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parameter sweep</a></span><ul class="toc-item"><li><span><a href="#Save-the-data" data-toc-modified-id="Save-the-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Save the data</a></span></li></ul></li></ul></div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.cell_models import cell_v1

import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, interact_manual

from statsmodels.tsa.stattools import acf

from scipy.signal import find_peaks

In [ ]:
from src.plot_utils import plot_vars_vs_time, plot_phase_RB, plot_phase_RBc, plot_autocorrelations

In [ ]:
from src.analysis import get_cycle_stats

# Test runs

Build something with interact so that you can get a rough sense of how the system behaves. 

In [ ]:
def run_and_plot_test(
    alpha=2, beta0=3, delta=1, 
    gamma=.9, epsilon=.01, dt=.1, 
    division="timer", transition="size", 
    time_SG2=1e-1, transition_th=2., T=300
):
    
    cell = cell_v1(
        alpha=alpha, beta0=beta0, delta=delta, 
        gamma=gamma, epsilon=epsilon, dt=dt, 
        division=division, transition=transition, 
        time_SG2=time_SG2, transition_th=transition_th
    )

    cell.grow(T)
    
    fig, ax = plt.subplots(2, 2, figsize=(15, 10))
    t_vec = np.arange(T+1)

    plot_vars_vs_time(ax[0,0], cell)
    plot_phase_RB(ax[1,0], cell)
    plot_phase_RBc(ax[1,1], cell)
    
    periods = plot_autocorrelations(ax[0,1], cell, nlags=T/2, prominence=0)
    if any([p is None for p in periods]):
        print("Not periodic")
    else:
        if np.std(periods)/np.mean(periods) < .05:
            print(f"All signals with period approx. {np.mean(periods)}.")
        else:
            print("Not periodic")
    
    return cell, periods

In [ ]:
cell_test, periods= run_and_plot_test()

# Interact

In [ ]:
step=.1

In [ ]:
interact_manual(
    run_and_plot_test, 
    alpha=(.1, 10., step), 
    beta0=(.001, .005, .001), 
    epsilon=(.1, 1., step), 
    gamma=(.01, .05, .01), 
    time_SG2 = fixed(12),#(1, 15), 
    transition_th=fixed(1.5),#(.1, 3, .1), 
    delta=fixed(1.),
    dt=fixed(1e-3), 
    division=fixed("timer"), 
    transition=fixed("size"), 
    T=fixed(100000)
);

---------

-----

# Parameter sweep

In [ ]:
from itertools import product

In [ ]:
# run a parameter sweep
T=8000
division="timer"
transition="size"
dt=1e-3
time_SG2 = .3
delta=1.

alpha_vals = np.arange(.1, 5., .5)
beta_vals = np.arange(.1, 5., .5)
epsilon_vals = np.arange(.1, 1, .2)
gamma_vals = np.arange(.1, 2, .3)
transition_th_vals = np.arange(.5, 3, .2)

In [ ]:
stats_tot = dict()
for alpha, beta0, epsilon, gamma, transition_th in product(
    alpha_vals, beta_vals, epsilon_vals, gamma_vals, transition_th_vals
):

    cell = cell_v1(alpha=alpha, beta0=beta0, delta=delta, 
            gamma=gamma, epsilon=epsilon, dt=dt, 
            division=division, transition=transition, 
            time_SG2=time_SG2, transition_th=transition_th)


    cell.grow(T)
    
    stats_crt = get_cycle_stats(cell)
    stats_tot[(alpha, beta0, epsilon, gamma, transition_th)] = stats_crt

## Save the data

In [ ]:
import pickle

In [ ]:
with open("data/stats_model_v1", 'wb') as f:
    pickle.dump(stats_tot, f)